In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from pennylane import numpy as pnp

## 3.1 Carregar Dados do Bloco Anterior

In [ ]:
# TODO: Carregar dados processados
# X_train = np.load('../data/X_train.npy')
# X_test = np.load('../data/X_test.npy')
# y_train = np.load('../data/y_train.npy')
# y_test = np.load('../data/y_test.npy')

# Converter labels para {-1, +1}
# y_train_quantum = 2 * y_train - 1
# y_test_quantum = 2 * y_test - 1

### Por que {-1, +1}?

<!-- TODO: Explicar escolha de labels -->
<!-- - PauliZ retorna valores em [-1, 1] -->
<!-- - Alinhamento com expectation values -->
<!-- - Facilita interpretação -->

## 3.2 Configuração do Quantum Device

In [ ]:
# TODO: Criar device
n_qubits = 2

## 3.3 Definição do QNode

O QNode conecta o circuito quântico ao framework de otimização.

In [ ]:
# TODO: Importar funções do notebook anterior
# (ou re-implementar angle_embedding e variational_circuit)

# TODO: Criar QNode
# @qml.qnode(dev, diff_method='parameter-shift')
# def quantum_circuit(params, x):
#     """
#     Complete quantum circuit with embedding + variational layers.
#     
#     Returns:
#         Expectation value of PauliZ on first qubit
#     """

### Como funciona o QNode?

<!-- TODO: Explicar pipeline -->
<!-- - Input clássico x -->
<!-- - Embedding → Variational layers -->
<!-- - Measurement (PauliZ) -->
<!-- - Output: expectation value ∈ [-1, 1] -->

## 3.4 Função de Previsão

In [ ]:
# TODO: Implementar função de previsão
def predict(params, x):
    """
    Predict class label for input x.
    
    Args:
        params: Circuit parameters
        x: Input data point
    
    Returns:
        Predicted label: -1 or +1
    """


## 3.5 Loss Function: MSE

In [ ]:
# TODO: Implementar loss function
def loss_function(params, X, y):
    """
    Mean Squared Error loss.
    
    Args:
        params: Circuit parameters
        X: Input data batch
        y: True labels (in {-1, +1})
    
    Returns:
        MSE loss
    """


### Por que MSE?

<!-- TODO: Justificar escolha -->
<!-- - Função diferenciável -->
<!-- - Funciona bem com expectation values -->
<!-- - Alternativas: hinge loss, cross-entropy -->

## 3.6 Quantum Gradients: Parameter-Shift

Demonstração de como o PennyLane calcula gradientes quânticos.

In [ ]:
# TODO: Demonstrar parameter-shift

# # Calcular gradiente

### Como funciona?

<!-- TODO: Explicar parameter-shift rule -->
<!-- - Avalia circuito com θ + π/2 e θ - π/2 -->
<!-- - Diferença das avaliações = gradiente -->
<!-- - PennyLane faz isso automaticamente -->

## 3.7 Loop de Treinamento

In [ ]:
# TODO: Configurar treinamento
n_layers = 2
n_epochs = 100
learning_rate = 0.1
batch_size = 10

# Inicializar parâmetros
# params = pnp.random.random((n_layers, n_qubits, 3), requires_grad=True)

# Otimizador
# opt = qml.GradientDescentOptimizer(stepsize=learning_rate)

# TODO: Loop de treinamento
# loss_history = []

# for epoch in range(n_epochs):
#     # Shuffle dados
#     
#     # Mini-batch training
#     # for i in range(0, len(X_train), batch_size):
#     
#     # Calcular loss da época
#     

## 3.8 Visualização do Treinamento

In [ ]:
# TODO: Plotar loss vs epochs
# plt.figure(figsize=(10, 6))
# plt.plot(loss_history, linewidth=2)
# plt.xlabel('Epoch', fontsize=12)
# plt.ylabel('Loss (MSE)', fontsize=12)
# plt.title('Training Loss Over Time', fontsize=14)
# plt.grid(True, alpha=0.3)
# plt.show()

## 3.9 Salvar Modelo Treinado

In [ ]:
# TODO: Salvar parâmetros otimizados
# np.save('../results/trained_params.npy', params)
# np.save('../results/loss_history.npy', loss_history)

## Checklist de Entrega

- [ ] QNode implementado corretamente
- [ ] Função de previsão funcionando
- [ ] Loss function (MSE) implementada
- [ ] Demonstração de parameter-shift
- [ ] Loop de treinamento completo
- [ ] Gráfico de loss vs epochs
- [ ] Parâmetros salvos para avaliação
- [ ] Slides preparados (treino + otimização)